# Práctica 3: Clasificación jerárquica
### Carlos Checa Moreno
### i02chmoc@uco.es
Cuaderno Google Colab: https://colab.research.google.com/drive/1cUB67oeCzo0BzhF9w8lexJTJYU8Zih3m?usp=sharing



Objetivo: El objetivo de esta práctica es introducir los conceptos de clasificación jerárquica

La práctica se puede realizar siguiendo una de las dos opciones siguientes:

## OPCIÓN 2: Comparación de métodos
Seleccione al menos dos algoritmos de los disponibles en la bibliotecas indicadas. Seleccione al menos tres problemas de clasificación jerárquica de los repositorios indicados.
Realice las siguientes tareas:
1. Aplique los algoritmos seleccionados a los datasets
2. Compare los resultados y explique qué conclusiones se podrían obtener

### Instalación librería

Usaré la librería [hiclass](https://github.com/scikit-learn-contrib/hiclass) aportada en los apuntes de la asignatura.

In [ ]:
pip install hiclass

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from hiclass import LocalClassifierPerNode, LocalClassifierPerParentNode
from hiclass.metrics import f1, precision, recall

### Descarga dataset

In [ ]:
!wget -O complaints.csv.zip 'https://files.consumerfinance.gov/ccdb/complaints.csv.zip'

--2025-02-28 15:52:20--  https://files.consumerfinance.gov/ccdb/complaints.csv.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 18.160.46.26, 18.160.46.121, 18.160.46.39, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|18.160.46.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1047695442 (999M) [binary/octet-stream]
Saving to: ‘complaints.csv.zip’

complaints.csv.zip  100%[===================>] 999.16M   128MB/s    in 9.6s    

2025-02-28 15:52:29 (105 MB/s) - ‘complaints.csv.zip’ saved [1047695442/1047695442]



### Preprocesado de Datos

A continuación, se carga el dataset, se eliminan valores nulos y se toma una muestra del 1%.

He decidido quedarme con una fracción del dataset para reducir el tiempo de ejecución ya que es un dataset considerablemente grande y los algoritmos que se utilizarán requieren un gran cantidad de memoria.

In [ ]:
'''------------ CARGA DEL DATASET ---------------'''
data = pd.read_csv(
    'complaints.csv.zip',
    compression='zip',
    header=0,
    sep=',',
    usecols=["Consumer complaint narrative", "Product", "Sub-product"]
)
data.dropna(inplace=True)
data = data.sample(frac=0.1, random_state=42)

X = data["Consumer complaint narrative"].to_numpy()
y = data[["Product", "Sub-product"]].to_numpy()

### División del Conjuto de Datos

Divido los datos de entrenamiento en 70% entrenamienot y 30% test.


In [ ]:
'''------------ DIVISIÓN DEL CONJUNTO DE DATOS ---------------'''
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Definición de Modelos

Usaré Regresión Logística como clasificador base, con un número alto de iteraciones para asegurar convergencia.

Como modelos a comparar he seleccionado: **LCPN** (Local Classifier Per Node) que entrena un clasificador por cada nodo de la jerarquía y **LCPPN** (Local Classifier Per Parent Node), el cual entrena un clasificador por cada nodo padre en la jerarquía.

In [ ]:
'''------------ DEFINICIÓN DE LOS MODELOS ---------------'''
# Se usa un clasificador base de Regresión Logística para ambos modelos.
base_classifier = LogisticRegression(random_state=42, max_iter=10000, n_jobs=1)

# Se definen los dos modelos de clasificación jerárquica a evaluar.
models = {
    "LCPN": LocalClassifierPerNode(local_classifier=base_classifier, verbose=0, n_jobs=1),
    "LCPPN": LocalClassifierPerParentNode(local_classifier=base_classifier, verbose=0, n_jobs=1)
}

### Entrenamiento

In [ ]:
'''------------ ENTRENAMIENTO Y EVALUACIÓN DE LOS MODELOS ---------------'''
# Evaluar cada modelo
results = {}
for name, model in models.items():
    print(f"Entrenando {name}...")
    pipeline = Pipeline([
        ('count', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('model', model),
    ])

    start_time = time.time()
    pipeline.fit(X_train, y_train)
    training_time = time.time() - start_time

    predictions = pipeline.predict(X_test)

    results[name] = {
        "Tiempo de entrenamiento": training_time,
        "F1-score": f1(y_test, predictions),
        "Precisión": precision(y_test, predictions),
        "Recall": recall(y_test, predictions)
    }

Entrenando LCPN...


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Entrenando LCPPN...


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


### Evaluación

In [ ]:
# Mostrar resultados
for model, metrics in results.items():
    print(f"Resultados de {model}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")
    print("\n")

Resultados de LCPN:
Tiempo de entrenamiento: 250.35141396522522
F1-score: 0.7519215675661046
Precisión: 0.7519215675661046
Recall: 0.7519215675661046


Resultados de LCPPN:
Tiempo de entrenamiento: 451.2287962436676
F1-score: 0.759465750087959
Precisión: 0.759465750087959
Recall: 0.759465750087959




### Interpretación resultados

<table>
        <tr>
            <th>Modelo</th>
            <th>Tiempo de Entrenamiento</th>
            <th>F1-score</th>
            <th>Precisión</th>
            <th>Recall</th>
        </tr>
        <tr>
            <td>LCPN</td>
            <td>250.35 s</td>
            <td>0.7519</td>
            <td>0.7519</td>
            <td>0.7519</td>
        </tr>
        <tr>
            <td>LCPPN</td>
            <td>451.23 s</td>
            <td>0.7595</td>
            <td>0.7595</td>
            <td>0.7595</td>
        </tr>
</table>

Analizando los resultados obtenidos:

*  El modelo **LCPN** ha requerido 250,35 segundos para entrenarse, obteniendo métricas de rendimiento equilibradas con F1-score, Precisión y Recall de 0.7519. Esto indica que el modelo logra una clasificación adecuada, aunque con margen de mejora.

*  El modelo **LCPPN**, por otro lado, ha necesitado 451.23 segundos para entrenarse, es decir, casi el doble de tiempo. Aunque ha logrado un desempeño ligeramente superior, con un F1-score, Precisión y Recall de 0.7595.

El aumento en el tiempo de entrenamiento del modelo LCPPN se justifica por la estrategia que utiliza para entrenar un clasificador por cada nodo padre en la jerarquía. A cambio, ofrece una ligera mejora en las métricas de rendimiento. Sin embargo, no considero que el aumento del desempeño sea suficiente para amortizar el aumento de tiempo de cómputo.

Si el tiempo de entrenamiento no es una restricción crítica, LCPPN podría ser una mejor opción. Sin embargo, si se busca un balance entre eficiencia y precisión, LCPN sería una mejor para este dataset.